In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import pickle

import numpy as np
from scipy import interpolate

sys.path.append(os.environ['DM21CM_DIR'])
import dm21cm.physics as phys

sys.path.append(os.environ['DH_DIR'])
from darkhistory.main import evolve as dh_evolve
import darkhistory.physics as dhphys

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../matplotlibrc')

## 1. DarkHistory runs

### 1.1 regular run

In [2]:
from darkhistory.config import load_data

In [3]:
! ls /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_debug

binning.p			 f_phot_swave_std_einasto_subs.p
boost_data.txt			 f_phot_swave_std.p
boost_Einasto_subs.txt		 hci_interp.raw
CMB_engloss_interp.raw		 hep_lb_interp.raw
CMB_limits_elec_decay.csv	 highengdep_interp.raw
CMB_limits_elec_swave.dat	 highengphot_tf_interp.raw
CMB_limits_phot_decay.csv	 ics_rel_ref_tf.raw
CMB_limits_phot_swave.dat	 ics_thomson_ref_tf.raw
dlNdlxIEW_coords_table.txt	 lci_interp.raw
dlNdlxIEW_values_table.txt	 lowengelec_tf_interp.raw
engloss_ref_tf.raw		 lowengphot_tf_interp.raw
f_elec_decay_std.p		 nntf_models
f_elec_swave_std_einasto_subs.p  std_soln_He.p
f_elec_swave_std.p		 tf_E_interp.raw
f_phot_decay_std.p


In [4]:
tf_name = 'debug'
dep_tf = load_data('dep_tf', prefix=f'/n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_{tf_name}', force_reload=True)

****** Loading transfer functions... ******
Using data at /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1_debug
    for propagating photons...  

In [ ]:
run_name = 'phph_debug_fine'

soln = dh_evolve(
    DM_process='swave', mDM=1e10, sigmav=1e-24, primary='phot_delta',
    start_rs=2965.0894801243167, end_rs=5,
    coarsen_factor=12, backreaction=True, 
    struct_boost=phys.struct_boost_func('erfc 1e-3'),
    reion_switch=False,
    use_tqdm=True, verbose=1,
    custom_dlnz=True, debug=True
)

In [6]:
pickle.dump(soln, open(f'../data/run_info/DH/{run_name}_soln.p', 'wb'))

### 1.2 force case A with fake reionization

In [ ]:
run_name = 'phph_caseA_in_reion'

soln = dh_evolve(
    DM_process='swave', mDM=1e10, sigmav=1e-23, primary='phot_delta',
    start_rs=3000, end_rs=6,
    coarsen_factor=12, backreaction=True, 
    struct_boost=phys.struct_boost_func('erfc 1e-3'),
    reion_switch=True, reion_rs=base_21cmfast['z'][1]+1,
    photoion_rate_func = [lambda rs: 0.] * 3,
    photoheat_rate_func = [lambda rs: 0.] * 3,
    use_tqdm=True, verbose=1
)

In [ ]:
pickle.dump(soln, open(f'../../data/run_info/DH/{run_name}_soln.p', 'wb'))

### 1.3 custom injection

In [ ]:
import darkhistory.physics as dhphys
from darkhistory.spec import pppc
from config import load_data

In [ ]:
# custom xe levels
def x_reion_func(rs):
    # extrapolate z low so that darkhistory doesn't complain
    z_arr = base_records['z'].copy()
    z_arr = np.append(z_arr, 2 * z_arr[-1] - z_arr[-2])
    x_e_arr = base_records['x_e'].copy()
    x_e_arr = np.append(x_e_arr, x_e_arr[-1])
    return interpolate.interp1d(1 + z_arr, x_e_arr)(rs)

In [ ]:
run_name = 'phph_turnon_z35_comp'

mDM = 1e10
sigmav = 1e-23
struct_boost = phys.struct_boost_func('erfc 1e-3')

photeng = load_data('binning')['phot']
eleceng = load_data('binning')['elec']

def in_spec_elec_func(rs):
    in_spec_elec = pppc.get_pppc_spec(1e10, eleceng, 'phot_delta', 'elec')
    in_spec_elec.rs = rs
    if rs > 36:
        in_spec_elec *= 0
    return in_spec_elec

def in_spec_phot_func(rs):
    in_spec_phot = pppc.get_pppc_spec(1e10, photeng, 'phot_delta', 'phot')
    in_spec_phot.rs = rs
    if rs > 36:
        in_spec_phot *= 0
    return in_spec_phot

def rate_func_N(rs):
    return (
        dhphys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)
        * struct_boost(rs) / (2*mDM)
    )
def rate_func_eng(rs):
    return (
        dhphys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav) 
        * struct_boost(rs)
    )

In [ ]:
soln = main.evolve(
    in_spec_elec = in_spec_elec_func,
    in_spec_phot = in_spec_phot_func,
    rate_func_N = rate_func_N,
    rate_func_eng = rate_func_eng,
    start_rs=3000, end_rs=7,
    coarsen_factor=12, backreaction=True, reion_switch=True,
    struct_boost=phys.struct_boost_func('erfc 1e-3'),
    verbose=1,
    use_tqdm=False
)

### 1.1 Get recfast values

In [ ]:
run_name = 'base_recfast'

rs_s = np.geomspace(start_rs, end_rs, 100)
x_e_s = dhphys.xHII_std(rs_s)
T_k_s = dhphys.Tm_std(rs_s) / phys.kB

arr_records = {
    'z' : rs_s - 1,
    'x_e' : x_e_s,
    'T_k' : T_k_s,
}
print(run_name)

In [ ]:
np.save(f'../../data/run_info/DH/{run_name}_records', arr_records)

## 2. Make arr_records and save

In [7]:
run_name = 'phph_debug_fine'
soln = pickle.load(open(f'../data/run_info/DH/{run_name}_soln.p', 'rb'))

In [8]:
arr_records = {
    'z' : soln['rs'] - 1,
    'x_e' : soln['x'][:,0],
    'T_k' : soln['Tm']/phys.kB,
    'f_heat' : soln['f']['low']['heat'] + soln['f']['high']['heat'],
    'f_ion' : soln['f']['low']['H ion']  + soln['f']['high']['H ion'] + \
              soln['f']['low']['He ion'] + soln['f']['high']['He ion'],
    'f_exc' : soln['f']['low']['exc'] + soln['f']['high']['exc'],
}

z_low, z_high = 6, 44
z_arr = arr_records['z'][::-1]
i_low = np.searchsorted(z_arr, z_low)
i_high = np.searchsorted(z_arr, z_high)

for k in arr_records.keys():
    arr_records[k] = arr_records[k][::-1][i_low:i_high]
    
print(run_name)

phph_debug_fine


In [9]:
np.save(f'../data/run_info/DH/{run_name}_records', arr_records)

## debug

In [150]:
from darkhistory.spec.spectrum import Spectrum

In [162]:
soln['f']['high']['heat'][342:400]

array([9.91152406e-04, 1.17104841e-01, 1.21482896e-01, 1.26428213e-01,
       1.32247690e-01, 1.39398441e-01, 1.48519366e-01, 1.60494729e-01,
       1.76500550e-01, 1.98059125e-01, 2.27106763e-01, 2.66028255e-01,
       3.17742954e-01, 3.85732988e-01, 4.74108116e-01, 5.87661937e-01,
       7.31879033e-01, 9.13067118e-01, 1.13828368e+00, 1.41561699e+00,
       1.75389566e+00, 2.16302752e+00, 2.65395920e+00, 3.23871124e+00,
       3.93049639e+00, 4.74434614e+00, 5.69538740e+00, 6.80153863e+00,
       8.08126883e+00, 9.55615359e+00, 1.12468571e+01, 1.31791477e+01,
       1.53808052e+01, 1.78788863e+01, 2.07049307e+01, 2.38932338e+01,
       2.74809560e+01, 3.15081328e+01, 3.60175145e+01, 4.10541318e+01,
       4.66719981e+01, 5.29453643e+01, 5.99126068e+01, 6.76254184e+01,
       7.61991157e+01, 8.57011357e+01, 9.61367208e+01, 1.07787155e+02])

In [ ]:
arr_records

In [152]:
hed_tf = dep_tf['highengdep']

In [153]:
from dm21cm.utils import load_dict

In [154]:
abscs = load_dict('../data/abscissas/abscs_debug_230802.h5')

In [155]:
rs = 40
dlnz = np.log(1.05)
dt = dlnz / phys.hubble(rs)
hed_tf.get_val(0.1, 0.1, 40) * dt / abscs['photE'][:,None]

array([[0.   , 0.   , 0.001, 0.   ],
       [0.   , 0.   , 0.001, 0.   ],
       [0.   , 0.   , 0.001, 0.   ],
       ...,
       [0.   , 0.   , 0.001, 0.   ],
       [0.   , 0.   , 0.001, 0.   ],
       [0.   , 0.   , 0.001, 0.   ]])

In [156]:
N = np.zeros((500,))
N[400] = 1
s = Spectrum(abscs['photE'], N, spec_type='N')

In [157]:
s.toteng()

2589268681.802462

In [158]:
highengdep_arr = hed_tf.get_val(1, 1, rs)

In [159]:
highengdep_arr

array([[0.00000000e+00, 0.00000000e+00, 6.63966807e-22, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 7.17195321e-22, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 7.74691029e-22, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 2.93232839e-05, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.16740563e-05, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.42132841e-05, 0.00000000e+00]])

In [160]:
highengdep_at_rs = np.dot(
    np.swapaxes(highengdep_arr, 0, 1),
    s.N
)

In [161]:
highengdep_at_rs * dt / s.toteng()

array([0.   , 0.   , 0.001, 0.   ])